In [1]:
import requests
import feedparser
from bs4 import BeautifulSoup
import time
import re
from llama_index.core import VectorStoreIndex, Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

In [2]:
# Configure embeddings
Settings.embed_model = HuggingFaceEmbedding(
    model_name="intfloat/e5-base-v2"
)

print("✅ Embeddings configured correctly)")

✅ Embeddings configured correctly)


In [4]:
def extract_news_text_from_url(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        response = requests.get(url, headers=headers, timeout=15)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Remove unwanted items
        for element in soup(['script', 'style', 'nav', 'footer', 'aside', 'form']):
            element.decompose()
        
        # Search main content
        selectors = ['article', '.article-body', '.story-body', '.post-content', '.content']
        content = None
        for selector in selectors:
            content = soup.select_one(selector)
            if content:
                break
        
        if not content:
            content = soup.body
        
        if content:
            text = content.get_text(separator='\n', strip=True)
            lines = [line.strip() for line in text.split('\n') if line.strip()]
            cleaned_text = '\n'.join(lines)
            
            # Clean BBC content
            mas_leidas_pattern = r'Saltar Más leídas.*?Final de Más leídas'
            cleaned_text = re.sub(mas_leidas_pattern, '', cleaned_text, flags=re.DOTALL | re.IGNORECASE)
            
            haz_clic_pattern = r'Haz clic aquí.*$'
            cleaned_text = re.sub(haz_clic_pattern, '', cleaned_text, flags=re.DOTALL | re.IGNORECASE)
            
            return cleaned_text
        else:
            return "It was not possible to extract the content"
            
    except Exception as e:
        return f"Error: {str(e)}"

In [5]:
# Configure RSS and get news
rss_url = "http://feeds.bbci.co.uk/mundo/rss.xml"

print("Obtaining BBC News...")
feed = feedparser.parse(rss_url)

max_articles = len(feed.entries)

articles = []
for i, entry in enumerate(feed.entries[:max_articles]):
    print(f"Processing New {i+1}: {entry.title}")
    
    link = entry.get('link', '')
    if link:
        content = extract_news_text_from_url(link)
        
        articles.append({
            'title': entry.get('title', 'Sin título'),
            'link': link,
            'published': entry.get('published', ''),
            'content': content
        })
    
    time.sleep(0.1)  # Pause to avoid saturating the server

print(f"✅ {len(articles)} news were obtained")

Obtaining BBC News...
Processing New 1: Qué significa que EE.UU. retire a Colombia la certificación en la lucha contra el narcotráfico y qué tiene que ver la tensa relación entre Trump y Petro
Processing New 2: "El bombardeo ha sido demencial": la huida de miles de palestinos tras el inicio de la toma israelí de Ciudad de Gaza
Processing New 3: "Mira debajo del teclado": los mensajes con los que Tyler Robinson le confesó a su pareja que había disparado contra Charlie Kirk
Processing New 4: Muere a los 89 años Robert Redford, la estrella que deslumbró a Hollywood con su carisma
Processing New 5: Cómo Juancho, un caimán del Orinoco, juega desde Dallas un papel clave para impedir la extinción de su especie
Processing New 6: Los 7 cargos de los que acusan a Tyler Robinson, sospechoso de asesinar al comentarista conservador Charlie Kirk
Processing New 7: Por qué la sentencia a la cúpula de las FARC es un hito en Colombia (y la controversia que generan las sanciones impuestas)
Processing New

In [7]:
# Create documents for the index
documents = []
for article in articles:
    doc_text = f"Tittle: {article['title']}\nDate: {article['published']}\nContent: {article['content']}"
    
    document = Document(
        text=doc_text,
        metadata={
            'title': article['title'],
            'url': article['link'],
            'date': article['published']
        }
    )
    documents.append(document)

# Build vector index (SOLO para búsqueda, sin generación)
print("Building index for semantic search...")
index = VectorStoreIndex.from_documents(documents)
print("Index correctly built")

# Create search retriever
retriever = index.as_retriever(similarity_top_k=3)

Building index for semantic search...
Index correctly built


In [11]:
def generate_summary(text, max_length=500):
    # Split into paragraphs
    paragraphs = [p.strip() for p in text.split('\n') if p.strip()]
    
    # Priorize longer paragraphs
    if paragraphs:
        # Take the longest paragraphs
        longest_para = max(paragraphs, key=len)
        
        if len(longest_para) > 100:
            summary = longest_para
        else:
            # Combine first paragraphs
            summary = ' '.join(paragraphs[:2])
        
        # Shorten if necessary
        if len(summary) > max_length:
            sentences = summary.split('. ')
            if len(sentences) > 1:
                summary = '. '.join(sentences[:2]) + '.'
            else:
                summary = summary[:max_length] + '...'
        
        return summary
    
    return text[:max_length] + '...' if len(text) > max_length else text




def search_news(query):
    print(f"🔍 Searching: '{query}'")
    print("-" * 60)
    
    try:
        # Getting relevant news
        results = retriever.retrieve(query)
        
        if not results:
            print("❌ No relevant news found")
            return []
        
        # Delete duplicate news
        unique_results = {}
        for node in results:
            title = node.metadata.get('title', 'Sin título')
            if title not in unique_results:
                unique_results[title] = node
            else:
                # If already exists, maintain the highest score
                if node.score > unique_results[title].score:
                    unique_results[title] = node
        
        unique_nodes = list(unique_results.values())
        
        # Sort per score
        unique_nodes.sort(key=lambda x: x.score, reverse=True)
        
        print(f"✅ {len(unique_nodes)} relevant news found:\n")
        
        for i, node in enumerate(unique_nodes):
            title = node.metadata.get('title', 'Sin título')
            score = node.score
            url = node.metadata.get('url', 'N/A')
            date = node.metadata.get('date', 'N/A')
            
            print(f"📰 {i+1}. {title} (Similarity: {score:.3f})")
            print(f"🔗 URL: {url}")
            print(f"📅 Date: {date}")
            print(f"📝 Summary:")

            summary = generate_summary(node.text)
            print(f"{summary}")
            
            print("-" * 50)
        
        return unique_nodes
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return []

In [12]:
# Example of searchs
queries = [
    "noticias políticas",
    "economía y negocios",
    "deportes",
    "tecnología",
    "salud y medicina",
    "cambio climático",
    "últimas noticias"
]

for query in queries:
    search_news(query)
    print("\n" + "="*80 + "\n")

🔍 Searching: 'noticias políticas'
------------------------------------------------------------
✅ 2 relevant news found:

📰 1. "El capitalismo depredador no funcionó para muchos y ahora esas clases resentidas buscan revancha" (Similarity: 0.809)
🔗 URL: https://www.bbc.com/mundo/articles/c5y07xjj45vo?at_medium=RSS&at_campaign=rss
📅 Date: Fri, 05 Sep 2025 03:46:36 GMT
📝 Summary:
El multimillonario Warren Buffett  dijo: "Hay una guerra de clase, sí. Pero es mi clase, la de los ricos, quien la está librando, y estamos ganando". Esta frase parece sugerir que los ricos están dando la batalla mientras las clases menos adineradas están cruzadas de brazos, han perdido conciencia de clase y se han desmovilizado. ¿Es así? ¿Cómo ha pasado esto?
--------------------------------------------------
📰 2. "Pierdo US$100.000 al mes": los empresarios en EE.UU. afectados por los aranceles de Trump (Similarity: 0.804)
🔗 URL: https://www.bbc.com/mundo/articles/c5ygvxvypk9o?at_medium=RSS&at_campaign=rss
📅 Date

In [ ]:
# Interactive search
print("🔍 INTERACTIVE SEARCH MODE")
print("Escribe 'salir' para terminar\n")

while True:
    query = input("¿Qué quieres buscar en las noticias? ").strip()
    
    if query.lower() in ['salir', 'exit', 'quit', '']:
        print("👋 ¡Hasta luego!")
        break
    
    if query:
        search_news(query)
        print("\n" + "="*80 + "\n")

🔍 INTERACTIVE SEARCH MODE
Escribe 'salir' para terminar



¿Qué quieres buscar en las noticias?  Noticias de cine


🔍 Searching: 'Noticias de cine'
------------------------------------------------------------
✅ 2 relevant news found:

📰 1. Muere a los 89 años Robert Redford, la estrella que deslumbró a Hollywood con su carisma (Similarity: 0.812)
🔗 URL: https://www.bbc.com/mundo/articles/cvg9n49eg0ro?at_medium=RSS&at_campaign=rss
📅 Date: Tue, 16 Sep 2025 12:37:48 GMT
📝 Summary:
De inmediato compañeros y personas que habían actuado junto a él reaccionaron ante a muerte de un ícono del cine de Hollywood y un padrino de las producciones independientes.
--------------------------------------------------
📰 2. Muere Robert Redford: 8 películas que muestran su categoría como actor y director (Similarity: 0.796)
🔗 URL: https://www.bbc.com/mundo/articles/c5yk2w955gro?at_medium=RSS&at_campaign=rss
📅 Date: Tue, 16 Sep 2025 17:23:01 GMT
📝 Summary:
En 1976 Redford y Dustin Hoffman interpretaron a los periodistas Bob Woodward y Carl Bernstein respectivamente en este thriller político sobre cómo se destapó el escá